In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* Install Flask, TensorFlow, and other dependencies.

* Install ngrok to expose the Flask app.

In [3]:
!pip install flask tensorflow rasterio Pillow pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 90.9 MB/s eta 0:00:00


# Create the Flask API

In [22]:
import os
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf
import rasterio
from PIL import Image
import io
from pyngrok import ngrok, conf
import absl.logging
import logging

# Suppress TensorFlow and Flask warnings
absl.logging.set_verbosity(absl.logging.ERROR)
logging.getLogger('werkzeug').setLevel(logging.ERROR)

# Initialize Flask app
app = Flask(__name__)

# Define paths
UPLOAD_FOLDER = 'uploads/'
MODEL_PATH = '/content/drive/MyDrive/Computer_Vision_intern/week_3/water_segmentation_model.h5'

# Ensure upload folder exists
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Load the trained model
model = tf.keras.models.load_model(MODEL_PATH)

# Function to preprocess the image
def preprocess_image(image):
    with rasterio.open(io.BytesIO(image)) as img:
        image_data = img.read()
        image_data = np.transpose(image_data, (1, 2, 0))

    image_data = image_data / 10000.0

    image_data = image_data[np.newaxis, ...]

    return image_data

# Function to generate the predicted mask
def predict_mask(image):
    pred_mask = model.predict(image)
    pred_mask = (pred_mask > 0.5).astype(np.uint8)  # Convert to binary mask
    return pred_mask[0, :, :, 0]  # Remove batch dimension

# API endpoint for prediction
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected"}), 400

    image = file.read()

    try:
        # Preprocess the image
        image_data = preprocess_image(image)

        # Generate the predicted mask
        mask = predict_mask(image_data)

        # Convert the mask to a PNG image
        mask_image = Image.fromarray(mask * 255)
        mask_bytes = io.BytesIO()
        mask_image.save(mask_bytes, format='PNG')
        mask_bytes.seek(0)

        # Return the predicted mask as a response
        return mask_bytes.getvalue(), 200, {'Content-Type': 'image/png'}

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run the Flask app with ngrok
if __name__ == '__main__':

    ngrok.set_auth_token('2t96QHFZbWRUerDp2vrJtEVeC7e_6Yc9dh8UxywztMGAGxrXu')

    # Start ngrok and create a tunnel to the Flask app
    public_url = ngrok.connect(5000).public_url
    print(f" * Running on {public_url}")

    # Run the Flask app
    app.run()

 * Running on https://24b6-34-57-28-53.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


# Run the Flask App with ngrok

In [32]:
%cd /content/drive/MyDrive/Computer_Vision_intern/week_3

/content/drive/MyDrive/Computer_Vision_intern/week_3


In [33]:
!ls

app.py	data  water_segmentation_model.h5


# run this in the ngrok

curl -X POST -F "file=@D:/computer_vision_intern/week_3/data/images/0.tif" https://cd2b-34-57-28-53.ngrok-free.app/predict --output mask.png

In [34]:
!python app.py

2025-03-05 13:09:01.325428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741180141.343741   25137 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741180141.348967   25137 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 13:09:05.843413: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
 * Running on https://4d42-34-57-28-53.ngrok-free.app
 * Serving Flask app 'app'
 * Debug mode: off
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://1